# Annotate RBC-GEM

## Setup
### Import packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from rbc_gem_utils import (
    COBRA_CONFIGURATION,
    ROOT_PATH,
    MODEL_PATH,
    get_annotation_df,
    compare_tables,
    visualize_comparison,
    read_rbc_model,
    write_rbc_model,
    show_versions,
)

from rbc_gem_utils.util import split_string
# Display versions of last time notebook ran and worked
show_versions()

### Define configuration
#### COBRA Configuration

In [ ]:
COBRA_CONFIGURATION

## Load RBC-GEM model
### Current Version: 0.2.0

In [ ]:
old_model = read_rbc_model(filetype="xml")
old_model

## Set annotations on model

In [ ]:
replace_annotations = True
annotation_types = ["reactions", "metabolites", "genes"]
model = old_model.copy()

for annotation_type in annotation_types:
    df_annotation = pd.read_csv(
        f"{ROOT_PATH}{MODEL_PATH}/{annotation_type}.tsv", 
        sep="\t", 
        index_col=0,
        dtype=str,
    )
    for idx, row in df_annotation.set_index(annotation_type).iterrows():
        item = getattr(model, annotation_type).get_by_id(idx)
        values = {k: split_string(v) for k, v in row.dropna().to_dict().items()}
        values.update({k: v[0] for k, v in values.items() if len(v) == 1})
        if replace_annotations:
            item.annotation = values
        else:
            item.annotation.update(values)
df_annotation

### Compare before overwriting

In [ ]:
df_old_model = {}
for annotation_type in annotation_types:
    all_annotation_keys = set()
    for annotation in getattr(old_model, annotation_type).list_attr("annotation"):
        all_annotation_keys.update(list(annotation.keys()))
    # Get annotation df is agnostic to object type, swap ID column with the `annotation_type`
    df_old_model[annotation_type] = get_annotation_df(
        getattr(old_model, annotation_type), 
        sorted(list(all_annotation_keys))
    ).rename({"id": annotation_type}, axis=1)

df_new_model = {}
for annotation_type in annotation_types:
    all_annotation_keys = set()
    for annotation in getattr(model, annotation_type).list_attr("annotation"):
        all_annotation_keys.update(list(annotation.keys()))
    # Get annotation df is agnostic to object type, swap ID column with the `annotation_type`
    df_new_model[annotation_type] = get_annotation_df(
        getattr(model, annotation_type), 
        sorted(list(all_annotation_keys))
    ).rename({"id": annotation_type}, axis=1)

In [ ]:
annotation_type = "reactions"
df_comparision = compare_tables(
    df_old_model[annotation_type].set_index(annotation_type),
    df_new_model[annotation_type].set_index(annotation_type),
)
visualize_comparison(df_comparision)

In [ ]:
annotation_type = "metabolites"
df_comparision = compare_tables(
    df_old_model[annotation_type].set_index(annotation_type),
    df_new_model[annotation_type].set_index(annotation_type),
)
visualize_comparison(df_comparision)

In [ ]:
annotation_type = "genes"
df_comparision = compare_tables(
    df_old_model[annotation_type].set_index(annotation_type),
    df_new_model[annotation_type].set_index(annotation_type),
)
visualize_comparison(df_comparision)

## Export model

In [ ]:
write_rbc_model(model, filetype="all")